In [12]:
import pandas as pd
import numpy as np
import warnings 

warnings.filterwarnings(action = "ignore")
input_path1 = '/home/vivana/projects/ml-basics/housing/dataset/'

In [20]:
house_data = pd.read_csv(input_path1 + 'AmesHousing.csv')
test = pd.read_csv(input_path1 + 'test.csv')
copied_house_data = house_data.copy()
copied_house_data.columns = copied_house_data.columns.str.replace(' ', '')
copied_house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Order          2930 non-null   int64  
 1   PID            2930 non-null   int64  
 2   MSSubClass     2930 non-null   int64  
 3   MSZoning       2930 non-null   object 
 4   LotFrontage    2440 non-null   float64
 5   LotArea        2930 non-null   int64  
 6   Street         2930 non-null   object 
 7   Alley          198 non-null    object 
 8   LotShape       2930 non-null   object 
 9   LandContour    2930 non-null   object 
 10  Utilities      2930 non-null   object 
 11  LotConfig      2930 non-null   object 
 12  LandSlope      2930 non-null   object 
 13  Neighborhood   2930 non-null   object 
 14  Condition1     2930 non-null   object 
 15  Condition2     2930 non-null   object 
 16  BldgType       2930 non-null   object 
 17  HouseStyle     2930 non-null   object 
 18  OverallQ